# How to run custom functions

# 1.  Creating a Custom Function as a Runnable Using RunnableLambda
Using RunnableLambda lets you explicitly define any function as a Runnable that you can integrate directly within LangChain.

In [1]:
from langchain_core.runnables import RunnableLambda

# Define a custom function
def length_function(text):
    return len(text)

# Wrap it in RunnableLambda
length_runnable = RunnableLambda(length_function)

# Use it in your pipeline
result = length_runnable.invoke("Hello, LangChain!")
print(result)  # Outputs the length of the text

17


# 2. 2. Using the @chain Decorator for Custom Functions
The @chain decorator is a convenient way to define a function as a Runnable. When applied, it makes the function behave as a Runnable that can be easily invoked.

Example:

Here’s how to use @chain to define a custom function that takes a topic, generates a joke about it, and identifies the joke's subject:

In [6]:
from langchain_core.runnables import chain
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

# Define prompts
prompt1 = ChatPromptTemplate.from_template("Tell me a joke about {topic}")
prompt2 = ChatPromptTemplate.from_template("What is the subject of this joke: {joke}")

# Define a custom chain function
@chain
def custom_chain(topic):
    # Generate a joke
    joke = ChatOpenAI().invoke(prompt1.invoke({"topic": topic}))
    # Find the subject of the joke
    subject = ChatOpenAI().invoke(prompt2.invoke({"joke": joke.content}))
    return subject.content

# Run the custom chain
result = custom_chain.invoke("bears")
print(result)  # Outputs the subject of the joke about bears


Bears


# 3. Automatic Coercion of Custom Functions in Chains
LangChain supports automatic coercion, so you can use a lambda or a standard function directly in a chain without wrapping it. LangChain will treat it as a Runnable.

Example:
Here’s a chain
 that takes a topic, generates a story about it, and retrieves the first five characters:

In [ ]:
prompt = ChatPromptTemplate.from_template("Tell me a story about {topic}")
model = ChatOpenAI()

# Automatically coerces lambda into a Runnable
chain = prompt | model | (lambda x: x.content[:5])

result = chain.invoke({"topic": "y"})
print(result)  # Outputs the first five characters of the story


Once 


# 4. Handling Metadata and Streaming in Custom Functions
If your function needs to handle streaming or metadata, LangChain provides ways to structure that using RunnableConfig and RunnableGenerator.

In [8]:
from langchain_core.runnables import RunnableLambda, RunnableConfig
# Define a function that accepts metadata
def custom_metadata_function(text, config: RunnableConfig):
    # Process the text here with access to config if needed
    return text.upper()

# Wrap it with RunnableLambda
metadata_runnable = RunnableLambda(custom_metadata_function)

# Invoke with metadata
result = metadata_runnable.invoke("hello", {"tags": ["example-tag"]})
print(result)  # Outputs: HELLO


HELLO


# 5. Using RunnableGenerator for Streaming
If you need a streaming output, use RunnableGenerator to process and yield chunks. This is useful for handling large outputs or data that needs to be processed as it arrives.

In [ ]:
from langchain_core.runnables import RunnableGenerator

# Define the generator function to handle iterators or single strings
def custom_streaming_function(input_text):
    # Join the input if it's an iterator or iterable, else use as is
    if isinstance(input_text, (list, tuple)) or hasattr(input_text, '__iter__'):
        input_text = ''.join(input_text)
    for word in input_text.split():
        yield word

# Initialize the RunnableGenerator with the generator function
stream_runnable = RunnableGenerator(custom_streaming_function)

# Pass a single string to the stream method directly
for chunk in stream_runnable.stream("This is a streamed response"):
    print(chunk)


This
is
a
streamed
response
